In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [1]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    df1 = pd.read_csv('university_towns.txt', delimiter='|', header=None)
    df1.rename(columns={0: 'RegionName'}, inplace=True)
    a = df1.loc[df1['RegionName'].str.endswith('[edit]')]
    a['State'] = a['RegionName']
    data1 = pd.merge(df1, a, left_on='RegionName', right_on='RegionName', how='outer')
    data1.fillna(method='ffill', inplace=True)
    data1.drop(data1.loc[data1['RegionName'] == data1['State']].index, inplace=True)
    data1['State'] = data1['State'].str.replace(r'\[.*', '')
    data1['RegionName'] = data1['RegionName'].str.replace(r'\s*\(.*', '')
    data1.reset_index(inplace=True)
    data1.drop(['index'], axis=1, inplace=True)
    data1['abc'] = data1['RegionName']
    data1.drop('RegionName', axis=1, inplace=True)
    data1.rename(columns={'abc': 'RegionName'}, inplace=True)
    return data1
get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    data3 = pd.read_excel('gdplev.xls', skiprows=5)
    data3.rename(columns={'Unnamed: 4': 'Quarter', 'GDP in billions of chained 2009 dollars.1': 'GDP'}, inplace=True)
    data3.drop([0, 1], inplace=True)
    data3 = data3.loc[data3.loc[data3['Quarter'].str.startswith('2000')].index[0]:, ['Quarter', 'GDP']]
    res = [[0, 0]]
    for i in range(data3.shape[0] - 4):
        if i > res[-1][-1]:
            for j in range(i + 2, data3.shape[0] - 2):
                if (data3.iloc[i, 1] > data3.iloc[i + 1, 1]) & (data3.iloc[i + 1, 1] > data3.iloc[i + 2, 1]) & (data3.iloc[j, 1] < data3.iloc[j + 1, 1]) & (data3.iloc[j + 1, 1] < data3.iloc[j + 2, 1]):
                    res.append([i + 1, j + 2])
                    break
    start = data3.iloc[res[-1][0], 0]
    data3.iloc[res[-1][-1], 0]
    bottom = data3.set_index('Quarter').iloc[res[-1][0]:res[-1][-1]]['GDP'].argmin()
    return start
get_recession_start()

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    data3 = pd.read_excel('gdplev.xls', skiprows=5)
    data3.rename(columns={'Unnamed: 4': 'Quarter', 'GDP in billions of chained 2009 dollars.1': 'GDP'}, inplace=True)
    data3.drop([0, 1], inplace=True)
    data3 = data3.loc[data3.loc[data3['Quarter'].str.startswith('2000')].index[0]:, ['Quarter', 'GDP']]
    res = [[0, 0]]
    for i in range(data3.shape[0] - 4):
        if i > res[-1][-1]:
            for j in range(i + 2, data3.shape[0] - 2):
                if (data3.iloc[i, 1] > data3.iloc[i + 1, 1]) & (data3.iloc[i + 1, 1] > data3.iloc[i + 2, 1]) & (data3.iloc[j, 1] < data3.iloc[j + 1, 1]) & (data3.iloc[j + 1, 1] < data3.iloc[j + 2, 1]):
                    res.append([i + 1, j + 2])
                    break
    start = data3.iloc[res[-1][0], 0]
    data3.iloc[res[-1][-1], 0]
    bottom = data3.set_index('Quarter').iloc[res[-1][0]:res[-1][-1]]['GDP'].argmin()       
    return data3.iloc[res[-1][-1], 0]
get_recession_end()

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    data3 = pd.read_excel('gdplev.xls', skiprows=5)
    data3.rename(columns={'Unnamed: 4': 'Quarter', 'GDP in billions of chained 2009 dollars.1': 'GDP'}, inplace=True)
    data3.drop([0, 1], inplace=True)
    data3 = data3.loc[data3.loc[data3['Quarter'].str.startswith('2000')].index[0]:, ['Quarter', 'GDP']]
    res = [[0, 0]]
    for i in range(data3.shape[0] - 4):
        if i > res[-1][-1]:
            for j in range(i + 2, data3.shape[0] - 2):
                if (data3.iloc[i, 1] > data3.iloc[i + 1, 1]) & (data3.iloc[i + 1, 1] > data3.iloc[i + 2, 1]) & (data3.iloc[j, 1] < data3.iloc[j + 1, 1]) & (data3.iloc[j + 1, 1] < data3.iloc[j + 2, 1]):
                    res.append([i + 1, j + 2])
                    break
    start = data3.iloc[res[-1][0], 0]
    data3.iloc[res[-1][-1], 0]
    bottom = data3.set_index('Quarter').iloc[res[-1][0]:res[-1][-1]]['GDP'].argmin()    
    return bottom
get_recession_bottom()

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    df = pd.DataFrame(pd.Series(states), columns=['StatesName'])
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'State'}, inplace=True)
    data2 = pd.read_csv('City_Zhvi_AllHomes.csv')
    data2 = data2.replace(to_replace='NaN', value=np.NaN).convert_objects(convert_numeric=True)
    for i in range(2000, 2016):
        data2[str(i)+'q1'] = data2.loc[:, [str(i)+'-01', str(i)+'-02', str(i)+'-03']].mean(axis=1)
        data2[str(i)+'q2'] = data2.loc[:, [str(i)+'-04', str(i)+'-05', str(i)+'-06']].mean(axis=1)
        data2[str(i)+'q3'] = data2.loc[:, [str(i)+'-07', str(i)+'-08', str(i)+'-09']].mean(axis=1)
        data2[str(i)+'q4'] = data2.loc[:, [str(i)+'-10', str(i)+'-11', str(i)+'-12']].mean(axis=1)
    data2['2016q1'] = data2.loc[:, ['2016-01', '2016-02', '2016-03']].mean(axis=1)
    data2['2016q2'] = data2.loc[:, ['2016-04', '2016-05', '2016-06']].mean(axis=1)
    data2['2016q3'] = data2.loc[:, ['2016-07', '2016-08', '2016-09']].mean(axis=1)
    data2.drop(data2.columns[6:251], axis=1, inplace=True)
    data2.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1, inplace=True)
    data2 = pd.merge(df, data2, left_on='State', right_on='State', how='right')
    data2.drop('State', axis=1, inplace=True)
    data2.rename(columns={'StatesName': 'State'}, inplace=True)
    data2.sort_values(['State', 'RegionName'], inplace=True)
    data2.set_index(['State', 'RegionName'], inplace=True)    
    return data2
convert_housing_data_to_quarters()

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    data2 = convert_housing_data_to_quarters()
    data1 = get_list_of_university_towns()
    data3 = pd.read_excel('gdplev.xls', skiprows=5)
    data3.rename(columns={'Unnamed: 4': 'Quarter', 'GDP in billions of chained 2009 dollars.1': 'GDP'}, inplace=True)
    data3.drop([0, 1], inplace=True)
    data3 = data3.loc[data3.loc[data3['Quarter'].str.startswith('2000')].index[0]:, ['Quarter', 'GDP']]
    res = [[0, 0]]
    for i in range(data3.shape[0] - 4):
        if i > res[-1][-1]:
            for j in range(i + 2, data3.shape[0] - 2):
                if (data3.iloc[i, 1] > data3.iloc[i + 1, 1]) & (data3.iloc[i + 1, 1] > data3.iloc[i + 2, 1]) & (data3.iloc[j, 1] < data3.iloc[j + 1, 1]) & (data3.iloc[j + 1, 1] < data3.iloc[j + 2, 1]):
                    res.append([i + 1, j + 2])
                    break
    start = data3.iloc[res[-1][0], 0]
    data3.iloc[res[-1][-1], 0]
    bottom = data3.set_index('Quarter').iloc[res[-1][0]:res[-1][-1]]['GDP'].argmin() 
    bstart = data2.columns[data2.columns.get_loc(start) - 1]
    data2['Ratio'] = data2[bstart] / data2[bottom]
    data2 = pd.DataFrame(data2.loc[:, 'Ratio'])
    data1 = data1.set_index(['State', 'RegionName'])
    total = pd.merge(data1, data2, left_index=True, right_index=True, how='right', indicator=True)
    univ = (total.loc[total['_merge'] == 'both']).dropna()
    non_univ = (total.loc[total['_merge'] == 'right_only']).dropna()
    (s, p) = stats.ttest_ind(univ['Ratio'], non_univ['Ratio'])
    if p < 0.01:
        different = True
    else:
        different = False
    if univ['Ratio'].mean() < non_univ['Ratio'].mean():
        better = 'university town'
    else:
        better = 'non-university town'   
    return (True, p, better)
run_ttest()